<a href="https://colab.research.google.com/github/JhoanVillaNO/proyectoKaggle/blob/main/99%20-%20modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalacion de dependencias

In [1]:
# Instalación de dependencias necesarias para el análisis
%pip install pandas numpy scikit-learn catboost matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.5 MB/s eta 0:00:00


## Carga de archivos

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.34GB/s]


In [3]:
!unzip udea*.zip > /dev/null

In [5]:
import os
import pandas as pd
import numpy as np
df = pd.read_csv("train.csv")
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


## Selección de variables y división de datos
Definimos las variables categóricas y separamos los datos en entrenamiento, validación y prueba, igual que en el flujo original.

In [7]:
# Definir variables categóricas
categorical_features = [
    "E_VALORMATRICULAUNIVERSIDAD",
    "E_HORASSEMANATRABAJA",
    "F_ESTRATOVIVIENDA",
    "F_EDUCACIONPADRE",
    "F_EDUCACIONMADRE",
    "E_PRGM_DEPARTAMENTO",
    "F_TIENEINTERNET.1",
    "E_PAGOMATRICULAPROPIO",
    "E_PRGM_ACADEMICO"
 ]

# Asegurar que las variables categóricas sean string y sin nulos
for col in categorical_features:
    if col in df.columns:
        df[col] = df[col].astype(str).fillna('missing')

from sklearn.model_selection import train_test_split
y = df["RENDIMIENTO_GLOBAL"]
X = df.drop(columns=["RENDIMIENTO_GLOBAL", "ID", "PERIODO_ACADEMICO"] )
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.125, random_state=42, stratify=y_temp)

## Entrenamiento y evaluación del modelo CatBoost
Entrenamos el modelo CatBoost usando las variables categóricas y evaluamos el desempeño en validación y test.

In [8]:
# Forzar todas las columnas categóricas a string y sin nulos
from catboost import CatBoostClassifier
for col in categorical_features:
    if col in X_train.columns:
        X_train[col] = X_train[col].astype(str).fillna('missing')
    if col in X_val.columns:
        X_val[col] = X_val[col].astype(str).fillna('missing')
    if col in X_test.columns:
        X_test[col] = X_test[col].astype(str).fillna('missing')
# Forzar todas las columnas numéricas a float (si aplica)
for col in X_train.columns:
    if col not in categorical_features:
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
        X_val[col] = pd.to_numeric(X_val[col], errors='coerce')
        X_test[col] = pd.to_numeric(X_test[col], errors='coerce')
# Reemplazar nulos numéricos por la media
for col in X_train.columns:
    if col not in categorical_features:
        mean_val = X_train[col].mean()
        X_train[col] = X_train[col].fillna(mean_val)
        X_val[col] = X_val[col].fillna(mean_val)
        X_test[col] = X_test[col].fillna(mean_val)

# Entrenamiento del modelo CatBoost
cat_features_indices = [X_train.columns.get_loc(col) for col in categorical_features if col in X_train.columns]
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    verbose=100,
    loss_function="MultiClass",
    cat_features=cat_features_indices
 )
model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

# Evaluación en validación
from sklearn.metrics import accuracy_score, classification_report
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Accuracy validación: {val_accuracy:.4f}")
print(classification_report(y_val, y_val_pred))

# Evaluación en test
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Accuracy test: {test_accuracy:.4f}")
print(classification_report(y_test, y_test_pred))

0:	learn: 1.3618577	test: 1.3618552	best: 1.3618552 (0)	total: 7.83s	remaining: 2h 10m 25s
100:	learn: 1.1936349	test: 1.1914024	best: 1.1914024 (100)	total: 11m 13s	remaining: 1h 39m 57s
200:	learn: 1.1878050	test: 1.1873358	best: 1.1873358 (200)	total: 22m 5s	remaining: 1h 27m 48s
300:	learn: 1.1851418	test: 1.1861534	best: 1.1861534 (300)	total: 33m 4s	remaining: 1h 16m 48s
400:	learn: 1.1830628	test: 1.1856395	best: 1.1856352 (398)	total: 44m 10s	remaining: 1h 5m 59s
500:	learn: 1.1813177	test: 1.1852680	best: 1.1852679 (499)	total: 55m 12s	remaining: 54m 59s
600:	learn: 1.1795424	test: 1.1849805	best: 1.1849805 (600)	total: 1h 6m 20s	remaining: 44m 2s
700:	learn: 1.1778393	test: 1.1848347	best: 1.1848290 (697)	total: 1h 17m 24s	remaining: 33m 1s
800:	learn: 1.1761167	test: 1.1847055	best: 1.1847016 (796)	total: 1h 28m 33s	remaining: 22m
900:	learn: 1.1744110	test: 1.1846061	best: 1.1846000 (892)	total: 1h 39m 32s	remaining: 10m 56s
999:	learn: 1.1726946	test: 1.1845285	best: 1.184

## Kaggle Submission

In [9]:
# Carga de datos de prueba
test_data = pd.read_csv("test.csv")

### Procesamiento de variables (Igual que en entrenamiento)


In [10]:
# Variables categóricas
for col in categorical_features:
    if col in test_data.columns:
        test_data[col] = test_data[col].astype(str).fillna('missing')

# Variables numéricas
for col in test_data.columns:
    if col not in categorical_features and col not in ["ID", "PERIODO_ACADEMICO"]:
        test_data[col] = pd.to_numeric(test_data[col], errors='coerce')
        mean_val = X_train[col].mean() if col in X_train.columns else 0
        test_data[col] = test_data[col].fillna(mean_val)

In [11]:
X_test_kaggle = test_data.drop(columns=["ID", "PERIODO_ACADEMICO"], errors='ignore')

# Predicciones en el conjunto de test
predictions = model.predict(X_test_kaggle)
predictions = predictions.flatten() if hasattr(predictions, 'flatten') else predictions

# Crear archivo de submission
submission_df = test_data[["ID"]].copy()
submission_df["RENDIMIENTO_GLOBAL"] = predictions
submission_df.to_csv("submission.csv", index=False)
print("Archivo de submission generado: submission.csv")
print(submission_df.head())

Archivo de submission generado: submission.csv
       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545         medio-bajo
2  499179               alto
3  782980               bajo
4  785185               bajo


In [12]:
# Guardar el archivo de submission
submission_df.to_csv("submission_catBoost.csv", index=False)